In [17]:
import spacy
import pydot
#from itertools import *
import es_core_news_md

from pprint import pprint
from itertools import islice
#from PrintGraph import PrintGraph

In [18]:
# WARNING!!! 
# This line may take a few seconds (and a few RAM MB/GB)

nlp = es_core_news_md.load()

In [19]:
verbs = ['dar','estudiar','durar','nacer','nace','tener','Tiene','poseer','posee','vivir','vive','trabajar','trabaja','radicar','radica','morir','muere','casar','iniciar','librar','disponer']

In [20]:
class BuildServices:
    
    def __init__(self):
            pass
        
    def sameLevel(self,token):
        
        for child in token.children:
            prep = 'ADP__AdpType=Prep' in child.tag_.split('|')
            pron = 'PronType=Art' in child.tag_.replace('_','|').split('|')
            relative = 'PronType=Rel' in child.tag_.replace('_','|').split('|')
            if(prep or pron or child.pos_ == 'ADP' or child.tag_ == 'amod' or relative or child.pos_ == 'SCONJ'):
                return False
        
        return True
    
    
    def ChildRelative(self,token):
        
        for child in token.children:
            relative = 'PronType=Rel' in child.tag_.replace('_','|').split('|')
            if(relative or child.pos_ == 'SCONJ'):
                return True
        
        return False
    
    def isIn(self,aspects,token):
        
        for x in aspects:
            if(x.__str__() == token.__str__()):
                return True
        
        return False
        
    def buildNominal(self,token):
        
        aspects = [token]
        
        for child in token.children:
            
            relative = self.ChildRelative(child)
            
            if((child.dep_ == 'nsubj' and not relative) or child.pos_ == 'PUNCT' or child.pos_ == 'CONJ'):
                continue
            
            if(len(list(child.children)) > 0):
                childlist = self.buildNominal(child)
                
                if(self.sameLevel(child)):
                    for x in childlist:
                        aspects.append(x)
                
                elif(child.pos_ == 'ADJ' and token.pos_ == 'VERB'):                    
                    obj = self.getObj(token)
                    l = [obj,child]
                    aspects.append(l)
                    
                
                else:
                    aspects = list(filter(lambda x : x.__str__() != token.__str__(), aspects))
                    for form in childlist:
                        l= [token,form]
                        aspects.append(l)
                        
            
            elif(((child.pos_ == 'ADJ' and token.pos_ == 'NOUN') or child.pos_ == 'NOUN' or child.pos_ == 'ADV' or child.pos_ == 'PROPN') and not (child.pos_ == 'VERB')):
                aspects = list(filter(lambda x : x.__str__() != token.__str__(), aspects))
                l= [token,child]
                aspects.append(l)
            
            
            elif(child.pos_ == 'ADJ' or child.pos_ == 'VERB'):
                aspects.append([child])

        return aspects
    
    
    
    def buildSintagma(self,token):
        if(token.pos_ != 'VERB' or token.lemma_ in verbs):
            return self.buildNominal(token)
        
        else:
            return []
    
    
    def BuildNsubj(self,nsubj):
        s = []
        for child in nsubj.children:
            if(len(list(child.children)) == 0 or child.pos_ == 'PROPN' or not self.sameLevel(child)):
                s += self.BuildNsubj(child)
            
        if(nsubj.dep_ != 'nsubj'):
            s.append(nsubj)
            
        return s
    
    def BuildNsubjS(self,nsubj):
        s = ''
        for child in nsubj.children:
            if(len(list(child.children)) == 0 or child.pos_ == 'PROPN' or not self.sameLevel(child)):
                s += self.BuildNsubjS(child) + ' '
        
        if(nsubj.dep_ != 'nsubj'):
            s += nsubj.__str__() + ' '
            
        return s
    
    def fix(self,aspect):
    
        ans = []
        for elem in aspect:
            if isinstance(elem,list):
                l = self.fix(elem)
                for x in l:
                    ans.append(x)
            else:
                ans.append(elem)

        return ans
    
    def Rebuild(self,dic):
        for key in dic.keys():
            aspects = dic[key]
            for i in range(0,len(aspects)):
                if isinstance(aspects[i],list) and len(aspects[i]) > 1:
                    l = self.fix(aspects[i])
                    l.sort(key = lambda x : x.idx)
                    aspects[i] = l
                dic[key][i] = aspects[i]
        return dic
    
        
    

In [21]:
class GetServices:
    
    def __init__(self):
        pass
    
    def getNsubj(self):
        for token in self.sent:
            if token.dep_ == 'nsubj':
                return token
    
    def getRoot(self):
        for token in self.sent:
            if token.dep_ == 'ROOT':
                return token
    
    def getObj(self,token):
        for child in token.children:
            if(child.dep_ == 'iobj' or child.dep_ == 'obj'):
                return child
        
    
    def getArticle(self,pos_fine):        
        return 'Omitido'
    
    def getOmmited(self,token):
        if(token.pos_ != 'VERB'):
            for child in token.children:
                if(child.lemma_ == 'ser' or child.lemma_ == 'estar'):
                    return getArticle(child.tag_)

        return self.getArticle(token.tag_)
    

In [22]:
class AspectSentence(GetServices,BuildServices):
    
    def __init__(self,sentence):
        self.sent = sentence
    
    def extractModifiers(self):
        nsub = self.getNsubj()
        l = []
        
        if(nsub):
            anc = list(nsub.ancestors)[0]
            while(anc.dep_ != 'ROOT'):
                l += self.buildSintagma(anc)
                anc = list(anc.ancestors)[0]
            
            l += self.buildSintagma(anc)
            
            for child in nsub.children:
                if(child.pos_ != 'DET' and (len(list(child.children)) > 0) and child.pos_ != 'PROPN' and self.sameLevel(child)):
                    l += self.buildSintagma(child)

        else:
            anc = self.getRoot()
            l = self.buildSintagma(anc)
    
        return {nsub.__str__() + ' ' + self.BuildNsubjS(nsub) if nsub else self.getOmmited(anc): ([[nsub] + self.BuildNsubj(nsub)] + l) if nsub else l }
    
    
    def HaveEnt(self,l):
        for token in l:
            if(token.ent_type_ and token.ent_type_ != 'NE'):
                return token
        
        return None
    
        

In [23]:
def Work(sentence):
   
    aspect = AspectSentence(sentence)
    dic = aspect.extractModifiers()
    
    return dic

In [24]:
text = """Una computadora portátil, ordenador portátil o computador portátil, es un dispositivo informático que se puede mover o transportar con relativa facilidad. Los ordenadores portátiles son capaces de realizar la mayor parte de las tareas que realizan los ordenadores de escritorio, también llamados «de torre»,o simplemente pc, con similares capacidades y con la ventaja de su peso y tamaño reducidos. Además, también tienen la capacidad de operar por un período determinado por medio de baterías recargables, sin estar conectadas a una red eléctrica. """

def extract_aspects(text):
    doc = nlp(text)

    aspect_info = []
    last = None

    for sentence in doc.sents:
        dic = Work(sentence)
        build = BuildServices()
        dic = build.Rebuild(dic)
        aspect_info.append(dic)

    #ASPECTS:
    ans = []
    for dic in aspect_info:
        for key in dic.keys():
            for aspect in dic[key]:
                if(aspect.__str__() != 'Omitido'):
                    ans.append(aspect)

    return(ans)

    


In [26]:
%timeit
extract_aspects(text)

[[Una, computadora, portátil],
 [dispositivo, informático],
 [dispositivo, transportar],
 [dispositivo, mover, relativa, facilidad],
 [ordenador, portátil],
 [computador, portátil],
 [Los, ordenadores, portátiles],
 [capaces, realizar, mayor, parte],
 [capaces, realizar, parte, tareas, realizan],
 [también, llamados],
 [llamados, torre»,o],
 [llamados, simplemente, pc],
 [llamados, similares, capacidades],
 [llamados, capacidades, ventaja, peso],
 [llamados, capacidades, ventaja, tamaño, reducidos],
 Además,
 [también, tienen],
 [tienen, capacidad, operar, período],
 [tienen, capacidad, operar, determinado, por, medio, baterías],
 [tienen, capacidad, operar, determinado, baterías, recargables],
 [capacidad, conectadas]]

In [14]:
from loadxml import iterxml

In [16]:
data = open('spanish_corpus/spanishText_10000_15000').read()
l = list(iterxml(data))



<doc id="20888" title="Idioma alemán" nonfiltered="345" processed="345" dbindex="10344">


El alemán (Deutsch, en alemán) es una lengua indoeuropea perteneciente al grupo de las lenguas germánicas occidentales. Es también una de las lenguas más importantes del mundo y la que más hablantes nativos tiene en la Unión Europea. Los adjetivos que se refieren al habla alemana son germanoparlante, germanohablante o germanófon.

Se habla principalmente en Alemania, Austria, Liechtenstein, en dos tercios de Suiza, en dos tercios de la provincia del Tirol del Sur (en Italia), en dos pequeños cantones del este de Bélgica y en algunos pueblos fronterizos del condado de Jutlandia meridional (Nordschleswig) en Dinamarca.

En Luxemburgo, así como en las regiones francesas de Alsacia y Lorena, las poblaciones nativas hablan dialectos alemanes, y algunos incluso dominan el alemán estándar (especialmente en Luxemburgo), aunque en Alsacia y Lorena el francés ha reemplazado significativamente a los dialec

In [27]:
len(l)

4613

In [40]:
l.sort(key=lambda x: len(x.text.strip()), reverse=True)

In [42]:
reduce = l[:50]
joblib.dump(reduce, 'reducelist.gz')

['reducelist.gz']

In [43]:
del(l)

In [44]:
del(text)

In [45]:
len(reduce)

50

In [46]:
extract_aspects(reduce[0].text)

NameError: name 'getArticle' is not defined